# Dynamic Circuits - Code Laboratory

> **Section 3: Create Quantum Circuits** | [📚 README](./README.md)

## Quick API Reference

| API | Purpose | Syntax |
|-----|---------|--------|
| `if_test()` | Binary decision | `with qc.if_test((clbit, value)):` |
| `for_loop()` | Fixed iterations | `with qc.for_loop(range(n)):` |
| `while_loop()` | Condition-based | `with qc.while_loop((clbit, value)):` |
| `switch()` | Multi-way branch | `with qc.switch(creg) as case:` |
| `reset()` | Return to \|0⟩ | `qc.reset(qubit)` |

### ⚠️ Key Differences (EXAM!)

| Feature | Python `for` | Qiskit `for_loop()` |
|---------|-------------|---------------------|
| When | Compile-time | Runtime |
| Result | Unrolled (N gates) | Single ForLoopOp |
| Execution | N separate instructions | Loop on hardware |

In [ ]:
# ============================================================
# SETUP: Imports and Utility Functions
# ============================================================
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit import Parameter
import numpy as np

def show_circuit_info(qc, name="Circuit"):
    """Display circuit with stats."""
    print(f"{name}:")
    print(qc.draw())
    print(f"  Qubits: {qc.num_qubits}, Classical bits: {qc.num_clbits}")
    print(f"  Operations: {qc.size()}, Depth: {qc.depth()}")

print("✅ Setup complete!")

---

## 🔬 What are Dynamic Circuits?

**Definition**: A quantum circuit where future operations depend on **runtime** measurement outcomes.

### Key Use Cases

| Use Case | Description |
|----------|-------------|
| **Quantum Error Correction** | Syndrome measurement + correction |
| **Repeat-Until-Success** | Retry until desired outcome |
| **Quantum Teleportation** | Conditional X/Z corrections |
| **Adaptive Algorithms** | Modify based on intermediate results |

---

## 1. if_test() - Conditional Gate Execution

**Signature**: `with qc.if_test((classical_bit, value)):`

| Parameter | Type | Description |
|-----------|------|-------------|
| `condition` | tuple | `(classical_bit, value)` to check |
| `as else_:` | optional | Captures else branch |

In [ ]:
# ============================================================
# IF_TEST: Basic Usage
# ============================================================

qc = QuantumCircuit(2, 2)

# Create superposition and measure
qc.h(0)
qc.measure(0, 0)

# Conditional gate: X if bit 0 == 1
with qc.if_test((qc.clbits[0], 1)):
    qc.x(1)  # Only executes if c[0] == 1

qc.measure(1, 1)

# Verification
assert qc.num_qubits == 2
assert qc.num_clbits == 2
print("if_test() - Conditional Gate:")
print(qc.draw())
print("\n✅ X on q1 only if c[0] == 1")

### If-Else Structure: `as else_:`

In [ ]:
# ============================================================
# IF-ELSE PATTERN: as else_:
# ============================================================

qc_ifelse = QuantumCircuit(2, 1)

qc_ifelse.h(0)
qc_ifelse.measure(0, 0)

# If-Else pattern using 'as else_:'
with qc_ifelse.if_test((qc_ifelse.clbits[0], 1)) as else_:
    qc_ifelse.x(1)   # If c[0] == 1: apply X
with else_:
    qc_ifelse.z(1)   # Else (c[0] == 0): apply Z

# Verification
assert qc_ifelse.num_qubits == 2
print("If-Else Pattern:")
print(qc_ifelse.draw())
print("\n✅ 'as else_:' captures the else branch!")

### ⚠️ TRAP: c_if() vs if_test() (EXAM!)

| Feature | c_if() (Deprecated) | if_test() (Modern) |
|---------|---------------------|-------------------|
| **Else clause** | ❌ Not supported | ✅ `as else_:` |
| **Multiple gates** | One per call | Block of gates |
| **Complex conditions** | ❌ Limited | ✅ `expr` module |
| **Status** | ⚠️ Deprecated | ✅ Recommended |

In [ ]:
# ============================================================
# ⚠️ TRAP DEMO: c_if() vs if_test() COMPARISON
# ============================================================

# --- c_if() (DEPRECATED - know for exam!) ---
qc_legacy = QuantumCircuit(2, 1)
qc_legacy.h(0)
qc_legacy.measure(0, 0)
qc_legacy.x(1).c_if(qc_legacy.clbits[0], 1)  # Method on gate

print("c_if() - Deprecated (one gate per call):")
print(qc_legacy.draw())

# --- if_test() (MODERN - use this!) ---
qc_modern = QuantumCircuit(2, 1)
qc_modern.h(0)
qc_modern.measure(0, 0)
with qc_modern.if_test((qc_modern.clbits[0], 1)):
    qc_modern.x(1)  # Can have multiple gates!
    qc_modern.z(1)

print("\nif_test() - Modern (multiple gates allowed):")
print(qc_modern.draw())

print("\n⚠️ c_if(): gate.c_if(bit, val)")
print("✅ if_test(): with qc.if_test((bit, val)):")

---

## 2. for_loop() - Fixed Iterations

**Signature**: `with qc.for_loop(range(n)) as i:`

### ⚠️ CRITICAL EXAM TRAP: Runtime vs Compile-Time!

In [ ]:
# ============================================================
# ⚠️ TRAP DEMO: Python for vs Qiskit for_loop()
# ============================================================

# --- Python for (COMPILE-TIME - UNROLLED) ---
qc_python = QuantumCircuit(1)
for i in range(3):
    qc_python.x(0)  # Creates 3 SEPARATE gates!

print("Python for loop (compile-time, UNROLLED):")
print(qc_python.draw())
print(f"  Number of gates: {qc_python.size()}")  # Shows 3

# --- Qiskit for_loop (RUNTIME - SINGLE OP) ---
qc_qiskit = QuantumCircuit(1)
with qc_qiskit.for_loop(range(3)):
    qc_qiskit.x(0)  # Single ForLoopOp!

print("\nQiskit for_loop (runtime, SINGLE operation):")
print(qc_qiskit.draw())
print(f"  Number of operations: {qc_qiskit.size()}")  # Shows 1

# Verification
assert qc_python.size() == 3, "Python for creates 3 gates"
assert qc_qiskit.size() == 1, "Qiskit for_loop creates 1 operation"
print("\n⚠️ EXAM: Python for = 3 gates, Qiskit for_loop = 1 operation!")

In [ ]:
# ============================================================
# FOR_LOOP: Practical Example - Rotations
# ============================================================

qc_for_rot = QuantumCircuit(1)

# Apply incremental rotations
with qc_for_rot.for_loop(range(4)):
    qc_for_rot.rx(np.pi/4, 0)  # Each iteration adds π/4

print("Rotational for_loop:")
print(qc_for_rot.draw())
print(f"\nTotal rotation: 4 × π/4 = π radians")
print("✅ One ForLoopOp executes 4 times at runtime")

---

## 3. while_loop() - Condition-Based Iteration

**Signature**: `with qc.while_loop((classical_bit, value)):`

⚠️ **EXAM TRAP**: Condition checked **BEFORE** each iteration!

In [ ]:
# ============================================================
# WHILE_LOOP: Repeat-Until-Success Pattern
# ============================================================

qc_while = QuantumCircuit(1, 1)

# Initial superposition
qc_while.h(0)

# While c[0] == 0: keep trying until we get |1⟩
with qc_while.while_loop((qc_while.clbits[0], 0)):
    qc_while.measure(0, 0)  # Measure
    qc_while.reset(0)       # Reset to |0⟩
    qc_while.h(0)           # Try again

print("while_loop - Repeat Until Success:")
print(qc_while.draw())

# Verification
assert qc_while.num_qubits == 1
print("\n✅ Condition (c[0]==0) checked BEFORE each iteration")
print("⚠️ If condition false at start, loop never executes!")

---

## 4. switch() - Multi-Branch Control

**Signature**: `with qc.switch(classical_register) as case:`

In [ ]:
# ============================================================
# SWITCH: Multiple Discrete Cases
# ============================================================

qc_switch = QuantumCircuit(2, 2)

# Create Bell state and measure
qc_switch.h(0)
qc_switch.cx(0, 1)
qc_switch.measure([0, 1], [0, 1])

# Switch on 2-bit register value (0, 1, 2, or 3)
with qc_switch.switch(qc_switch.cregs[0]) as case:
    with case(0):  # Register == 00
        qc_switch.id(0)
    with case(1):  # Register == 01
        qc_switch.x(0)
    with case(2):  # Register == 10
        qc_switch.z(0)
    with case(3):  # Register == 11
        qc_switch.y(0)

print("switch() - Multi-Branch:")
print(qc_switch.draw())

# Verification
assert qc_switch.num_qubits == 2
print("\n✅ Different gate based on 2-bit measurement result")

In [ ]:
# ============================================================
# SWITCH: DEFAULT Case
# ============================================================

qc_default = QuantumCircuit(2, 2)

qc_default.h([0, 1])
qc_default.measure([0, 1], [0, 1])

# Switch with DEFAULT for unhandled cases
with qc_default.switch(qc_default.cregs[0]) as case:
    with case(0):  # Only handle 00
        qc_default.x(0)
    with case(case.DEFAULT):  # All other (01, 10, 11)
        qc_default.h(0)

print("switch() with DEFAULT:")
print(qc_default.draw())
print("\n✅ case.DEFAULT handles all unspecified cases")

---

## 5. expr Module - Complex Classical Conditions (EXAM!)

**From README**: The `qiskit.circuit.classical.expr` module enables AND/OR/comparison logic.

| Function | Description | Example |
|----------|-------------|---------|
| `expr.logic_and(a, b)` | Logical AND | Both bits == 1 |
| `expr.logic_or(a, b)` | Logical OR | Either bit == 1 |
| `expr.equal(a, b)` | Equality check | Register == value |
| `expr.greater_equal(a, b)` | Greater or equal | Register >= value |

In [ ]:
# ============================================================
# expr MODULE: Complex Classical Conditions
# ============================================================

from qiskit.circuit.classical import expr

qc_expr = QuantumCircuit(3, 2)

# Setup: Create superposition and measure
qc_expr.h([0, 1])
qc_expr.measure([0, 1], [0, 1])

# LOGICAL AND: Both bits must be 1
condition_and = expr.logic_and(qc_expr.clbits[0], qc_expr.clbits[1])
with qc_expr.if_test(condition_and):
    qc_expr.x(2)  # X only if c[0]==1 AND c[1]==1

print("expr.logic_and() - Both bits must be 1:")
print(qc_expr.draw())

# Verification
assert qc_expr.num_qubits == 3
print("\n✅ X applied only when BOTH bits are 1")
print("📌 EXAM: Use expr module for complex conditions!")

In [ ]:
# ============================================================
# expr MODULE: Logical OR and Comparisons
# ============================================================

from qiskit.circuit.classical import expr
from qiskit import QuantumCircuit, ClassicalRegister

# Create circuit with named register for better clarity
cr = ClassicalRegister(2, 'c')
qc_or = QuantumCircuit(3, cr)

qc_or.h([0, 1])
qc_or.measure([0, 1], [0, 1])

# LOGICAL OR: Either bit can be 1
condition_or = expr.logic_or(cr[0], cr[1])
with qc_or.if_test(condition_or):
    qc_or.x(2)  # X if c[0]==1 OR c[1]==1

print("expr.logic_or() - Either bit is 1:")
print(qc_or.draw())
print("\n✅ X applied if EITHER bit is 1 (00 → no X, else → X)")

# ============================================================
# COMPARISON: greater_equal, equal
# ============================================================

qc_cmp = QuantumCircuit(3, cr)
qc_cmp.h([0, 1])
qc_cmp.measure([0, 1], [0, 1])

# Register >= 2 means binary '10' or '11'
condition_gte = expr.greater_equal(cr, 2)
with qc_cmp.if_test(condition_gte):
    qc_cmp.z(2)

print("\nexpr.greater_equal(cr, 2) - Register >= 2:")
print(qc_cmp.draw())
print("\n✅ Z applied when cr >= 2 (i.e., '10' or '11')")
print("\n📌 EXAM: Register comparison interprets bits as INTEGER")
print("   cr=2 → '10', cr=3 → '11', etc.")

### ⚠️ TRAP: expr Conditions vs Tuple Conditions (EXAM!)

| Syntax | Use Case | Example |
|--------|----------|---------|
| `(clbit, value)` | Simple single-bit check | `(qc.clbits[0], 1)` |
| `(register, value)` | Simple register equality | `(cr, 3)` |
| `expr.logic_and(...)` | Complex AND condition | Both bits == 1 |
| `expr.greater_equal(...)` | Comparison operations | Register >= value |

In [ ]:
# ============================================================
# ⚠️ TRAP DEMO: Tuple vs expr - When to Use Which
# ============================================================

from qiskit.circuit.classical import expr

# Create circuit
cr = ClassicalRegister(2, 'c')
qc_trap = QuantumCircuit(2, cr)
qc_trap.h([0, 1])
qc_trap.measure([0, 1], [0, 1])

# --- TUPLE SYNTAX (Simple cases) ---
# ✅ Check single bit
with qc_trap.if_test((cr[0], 1)):  # Simple: c[0] == 1
    qc_trap.id(0)  # Placeholder

# ✅ Check whole register equals value
with qc_trap.if_test((cr, 2)):  # Simple: cr == 2
    qc_trap.id(0)  # Placeholder

print("✅ Tuple syntax: Good for simple equality checks")

# --- expr SYNTAX (Complex cases) ---
# When you need AND, OR, or comparisons, MUST use expr

# ❌ You CANNOT do this with tuple:
# with qc.if_test((cr[0], 1) AND (cr[1], 1)):  # ERROR!

# ✅ Use expr for complex logic:
condition = expr.logic_and(cr[0], cr[1])  # Both bits == 1
condition2 = expr.greater_equal(cr, 2)    # cr >= 2

print("✅ expr module: Required for AND, OR, comparisons")

print("\n⚠️ EXAM TRAP:")
print("  Simple equality → Tuple: (bit, value)")
print("  Complex logic → expr module")

---

## 5. 🔴 Quantum Teleportation (GUARANTEED EXAM!)

In [ ]:
# ============================================================
# 🔴 QUANTUM TELEPORTATION - MEMORIZE THIS!
# ============================================================

qc = QuantumCircuit(3, 2)

# STEP 0: Prepare state to teleport
qc.h(0)  # Teleporting |+⟩
qc.barrier()

# STEP 1: Create Bell pair (qubits 1 and 2)
qc.h(1)
qc.cx(1, 2)
qc.barrier()

# STEP 2: Bell measurement (qubits 0 and 1)
qc.cx(0, 1)
qc.h(0)
qc.measure([0, 1], [0, 1])
qc.barrier()

# STEP 3: Conditional corrections (CRITICAL!)
with qc.if_test((qc.clbits[1], 1)):  # X if c[1]=1
    qc.x(2)
with qc.if_test((qc.clbits[0], 1)):  # Z if c[0]=1
    qc.z(2)

print("QUANTUM TELEPORTATION:")
print(qc.draw())

# Verification
assert qc.num_qubits == 3, "Need 3 qubits"
assert qc.num_clbits == 2, "Need 2 classical bits"
print("\n🎯 EXAM: X correction → c[1], Z correction → c[0]")
print("📌 MNEMONIC: 'X-1, Z-0' (alphabetically reversed!)")

### Teleportation Correction Table

| Measurement (c0, c1) | X Correction | Z Correction | Result |
|---------------------|--------------|--------------|--------|
| 00 | ❌ | ❌ | State teleported |
| 01 | ✅ | ❌ | State teleported |
| 10 | ❌ | ✅ | State teleported |
| 11 | ✅ | ✅ | State teleported |

**⚠️ TRAP**: X uses c[1], Z uses c[0] (NOT intuitive order!)

---

## 6. reset() - Qubit Reuse

**Signature**: `qc.reset(qubit)` - Forces qubit to |0⟩

In [ ]:
# ============================================================
# RESET: Force Qubit to |0⟩ and Reuse
# ============================================================

qc_reset = QuantumCircuit(1, 2)

# First use
qc_reset.h(0)
qc_reset.measure(0, 0)

# Reset to |0⟩
qc_reset.reset(0)

# Reuse qubit
qc_reset.x(0)
qc_reset.measure(0, 1)

print("Reset and Reuse:")
print(qc_reset.draw())

# Verification
assert qc_reset.num_qubits == 1
assert qc_reset.num_clbits == 2
print("\n✅ reset() returns qubit to |0⟩ for reuse")
print("📌 Use case: Qubit recycling in long circuits")

---

## 🧪 CODE CHALLENGES

In [ ]:
# ============================================================
# CHALLENGE 1: Python for vs Qiskit for_loop
# ============================================================
# How many gates does each create for 5 iterations?

# Python for:
qc_py = QuantumCircuit(1)
for i in range(5):
    qc_py.rx(np.pi/5, 0)

# Qiskit for_loop:
qc_qs = QuantumCircuit(1)
with qc_qs.for_loop(range(5)):
    qc_qs.rx(np.pi/5, 0)

# Your answers:
python_gates = 5       # How many gates in qc_py?
qiskit_ops = 1         # How many operations in qc_qs?

# ============================================================
# VERIFICATION
# ============================================================
assert qc_py.size() == python_gates, f"Python for: expected {python_gates}, got {qc_py.size()}"
assert qc_qs.size() == qiskit_ops, f"Qiskit for_loop: expected {qiskit_ops}, got {qc_qs.size()}"
print(f"✅ Challenge 1 PASSED!")
print(f"   Python for: {python_gates} gates (unrolled)")
print(f"   Qiskit for_loop: {qiskit_ops} operation (runtime)")

In [ ]:
# ============================================================
# CHALLENGE 2: Teleportation Corrections
# ============================================================
# Fill in the correct classical bits for X and Z corrections

qc = QuantumCircuit(3, 2)

# Teleportation setup (given)
qc.h(0)
qc.h(1)
qc.cx(1, 2)
qc.cx(0, 1)
qc.h(0)
qc.measure([0, 1], [0, 1])

# YOUR TASK: Fill in the correct bit indices!
x_correction_bit = 1  # Which bit controls X correction?
z_correction_bit = 0  # Which bit controls Z correction?

# Apply corrections
with qc.if_test((qc.clbits[x_correction_bit], 1)):
    qc.x(2)
with qc.if_test((qc.clbits[z_correction_bit], 1)):
    qc.z(2)

# ============================================================
# VERIFICATION
# ============================================================
assert x_correction_bit == 1, "X uses c[1]!"
assert z_correction_bit == 0, "Z uses c[0]!"
print("✅ Challenge 2 PASSED: Teleportation corrections correct!")
print("   X correction: c[1]")
print("   Z correction: c[0]")
print("   MNEMONIC: 'X-1, Z-0'")

In [ ]:
# ============================================================
# CHALLENGE 3: If-Else Structure
# ============================================================
# Create a circuit that applies X if c[0]==1, else Z

qc = QuantumCircuit(2, 1)

qc.h(0)
qc.measure(0, 0)

# YOUR SOLUTION:
with qc.if_test((qc.clbits[0], 1)) as else_:
    qc.x(1)   # If c[0] == 1
with else_:
    qc.z(1)   # Else (c[0] == 0)

# ============================================================
# VERIFICATION
# ============================================================
assert qc.num_qubits == 2
assert qc.num_clbits == 1
print("✅ Challenge 3 PASSED: If-else pattern correct!")
print(qc.draw())

---

## 📚 Resources

- [Qiskit Dynamic Circuits](https://docs.quantum.ibm.com/api/qiskit/circuit)
- [Dynamic Circuits Tutorial](https://learning.quantum.ibm.com/tutorial/dynamic-circuits)
- [if_test Documentation](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.if_test)

---

**Section 3: Create Quantum Circuits** | [← Classical Control](./classical_control.ipynb) | [README](./README.md)

*Code Laboratory v1.0 | Qiskit 1.x Compatible*